In [20]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
import pprint
import pyspark
import pyspark.sql.functions as F

from pyspark.sql.functions import col
from pyspark.sql.types import StringType, IntegerType, FloatType, DateType

import utils.data_processing_bronze_table
import utils.data_processing_silver_table

## Set up Spark session

In [2]:
# Initialize SparkSession
spark = pyspark.sql.SparkSession.builder \
    .appName("dev") \
    .master("local[*]") \
    .getOrCreate()

# Set log level to ERROR to hide warnings
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/05 02:17:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Set up Config

In [3]:
DATASETS = [
    {
        "name": "feature_clickstream",
        "csv_file_path": "data/feature_clickstream.csv",
        "bronze_partition": "bronze_features_clickstream_",
        "bronze_path": "datamart/bronze/features_clickstream/",
        "update_type": "append"
    },
    {
        "name": "features_attributes",
        "csv_file_path": "data/features_attributes.csv",
        "bronze_partition": "bronze_features_attribute_",
        "bronze_path": "datamart/bronze/features_attributes/",
        "update_type": "append"
    },
    {
        # Overwrite-at-source; we stamp each ingest with "today"
        "name": "features_financials",
        "csv_file_path": "data/features_financials.csv",
        "bronze_partition": "bronze_features_financials_",
        "bronze_path": "datamart/bronze/features_financials/",
        "update_type": "overwrite"
    },
    {
        "name": "lms_loan_daily",
        "csv_file_path": "data/lms_loan_daily.csv",
        "bronze_partition": "bronze_lms_loan_daily_",
        "bronze_path": "datamart/bronze/lms_loan_daily/",
        "update_type": "append"
    },
]

In [4]:
# generate list of dates to process
def generate_first_of_month_dates(start_date_str, end_date_str):
    # Convert the date strings to datetime objects
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
    
    # List to store the first of month dates
    first_of_month_dates = []

    # Start from the first of the month of the start_date
    current_date = datetime(start_date.year, start_date.month, 1)

    while current_date <= end_date:
        # Append the date in yyyy-mm-dd format
        first_of_month_dates.append(current_date.strftime("%Y-%m-%d"))
        
        # Move to the first of the next month
        if current_date.month == 12:
            current_date = datetime(current_date.year + 1, 1, 1)
        else:
            current_date = datetime(current_date.year, current_date.month + 1, 1)

    return first_of_month_dates

## Build Bronze Table

In [5]:
for file in DATASETS:

    csv_file_path = file["csv_file_path"]
    bronze_lms_directory = file["bronze_path"]
    bronze_partition = file["bronze_partition"]
    update_type = file["update_type"]

    if not os.path.exists(bronze_lms_directory):
        os.makedirs(bronze_lms_directory)

    if update_type=="append":
        df = pd.read_csv(file["csv_file_path"])
        min_date = df["snapshot_date"].min()
        max_date = df["snapshot_date"].max()
        dates_str_lst = generate_first_of_month_dates(min_date, max_date)

        # run bronze backfill
        for date_str in dates_str_lst:
            utils.data_processing_bronze_table.process_bronze_table_append(date_str, csv_file_path, bronze_partition, bronze_lms_directory, spark)

    else:
        min_date = pd.to_datetime(date.today()).strftime("%Y-%m-%d")
        max_date = min_date
        dates_str_lst = generate_first_of_month_dates(min_date, max_date)

        # run bronze backfill
        for date_str in dates_str_lst:
            utils.data_processing_bronze_table.process_bronze_table_overwrite(date_str, csv_file_path, bronze_partition, bronze_lms_directory, spark)

2023-01-01row count: 8974


saved to: datamart/bronze/features_clickstream/bronze_features_clickstream_2023_01_01.csv


2023-02-01row count: 8974


saved to: datamart/bronze/features_clickstream/bronze_features_clickstream_2023_02_01.csv


2023-03-01row count: 8974


...

saved to: datamart/bronze/lms_loan_daily/bronze_lms_loan_daily_2023_01_01.csv
2023-02-01row count: 1031
saved to: datamart/bronze/lms_loan_daily/bronze_lms_loan_daily_2023_02_01.csv
2023-03-01row count: 1537


saved to: datamart/bronze/lms_loan_daily/bronze_lms_loan_daily_2023_03_01.csv
2023-04-01row count: 2047
saved to: datamart/bronze/lms_loan_daily/bronze_lms_loan_daily_2023_04_01.csv
2023-05-01row count: 2568
saved to: datamart/bronze/lms_loan_daily/bronze_lms_loan_daily_2023_05_01.csv
2023-06-01row count: 3085
saved to: datamart/bronze/lms_loan_daily/bronze_lms_loan_daily_2023_06_01.csv
2023-07-01row count: 3556
saved to: datamart/bronze/lms_loan_daily/bronze_lms_loan_daily_2023_07_01.csv
2023-08-01row count: 4037
saved to: datamart/bronze/lms_loan_daily/bronze_lms_loan_daily_2023_08_01.csv
2023-09-01row count: 4491
saved to: datamart/bronze/lms_loan_daily/bronze_lms_loan_daily_2023_09_01.csv
2023-10-01row count: 4978
saved to: datamart/bronze/lms_loan_daily/bronze_lms_loan_daily_2023_10_01.csv


2023-11-01row count: 5469


saved to: datamart/bronze/lms_loan_daily/bronze_lms_loan_daily_2023_11_01.csv
2023-12-01row count: 5428
saved to: datamart/bronze/lms_loan_daily/bronze_lms_loan_daily_2023_12_01.csv
2024-01-01row count: 5412
saved to: datamart/bronze/lms_loan_daily/bronze_lms_loan_daily_2024_01_01.csv
2024-02-01row count: 5424
saved to: datamart/bronze/lms_loan_daily/bronze_lms_loan_daily_2024_02_01.csv
2024-03-01row count: 5425
saved to: datamart/bronze/lms_loan_daily/bronze_lms_loan_daily_2024_03_01.csv
2024-04-01row count: 5417
saved to: datamart/bronze/lms_loan_daily/bronze_lms_loan_daily_2024_04_01.csv
2024-05-01row count: 5391
saved to: datamart/bronze/lms_loan_daily/bronze_lms_loan_daily_2024_05_01.csv
2024-06-01row count: 5418
saved to: datamart/bronze/lms_loan_daily/bronze_lms_loan_daily_2024_06_01.csv
2024-07-01row count: 5442
saved to: datamart/bronze/lms_loan_daily/bronze_lms_loan_daily_2024_07_01.csv
2024-08-01row count: 5531
saved to: datamart/bronze/lms_loan_daily/bronze_lms_loan_daily_2

saved to: datamart/bronze/lms_loan_daily/bronze_lms_loan_daily_2024_12_01.csv
2025-01-01row count: 5539
saved to: datamart/bronze/lms_loan_daily/bronze_lms_loan_daily_2025_01_01.csv
2025-02-01row count: 5028
saved to: datamart/bronze/lms_loan_daily/bronze_lms_loan_daily_2025_02_01.csv
2025-03-01row count: 4515
saved to: datamart/bronze/lms_loan_daily/bronze_lms_loan_daily_2025_03_01.csv
2025-04-01row count: 4024
saved to: datamart/bronze/lms_loan_daily/bronze_lms_loan_daily_2025_04_01.csv
2025-05-01row count: 3526
saved to: datamart/bronze/lms_loan_daily/bronze_lms_loan_daily_2025_05_01.csv
2025-06-01row count: 3021
saved to: datamart/bronze/lms_loan_daily/bronze_lms_loan_daily_2025_06_01.csv
2025-07-01row count: 2478
saved to: datamart/bronze/lms_loan_daily/bronze_lms_loan_daily_2025_07_01.csv
2025-08-01row count: 1985
saved to: datamart/bronze/lms_loan_daily/bronze_lms_loan_daily_2025_08_01.csv
2025-09-01row count: 1529
saved to: datamart/bronze/lms_loan_daily/bronze_lms_loan_daily_2

## Build Silver Table

In [21]:
# create bronze datalake
silver_loan_daily_directory = "datamart/silver/features_attributes/"
bronze_lms_directory = "datamart/bronze/features_attributes/"
date_str = "2023-01-01"

if not os.path.exists(silver_loan_daily_directory):
    os.makedirs(silver_loan_daily_directory)

pd.set_option('display.max_rows', None)
utils.data_processing_silver_table.process_silver_table(date_str, bronze_lms_directory, silver_loan_daily_directory, spark).toPandas()

loaded from: datamart/bronze/features_attributes/bronze_features_attribute_2023_01_01.csv row count: 530
saved to: datamart/silver/features_attributes/silver_features_attribute_2023_01_01.parquet


,Customer_ID,Name,Age,SSN,Occupation,snapshot_date,Name_clean,Age_clean
0,CUS_0x1037,Matthewm,45.0,230-22-9583,Accountant,2023-01-01,Matthewm,45
1,CUS_0x1069,Andreas Cremero,32.0,761-27-5143,Accountant,2023-01-01,Andreas Cremero,32
2,CUS_0x114a,Valetkevitchu,43.0,133-89-5234,Developer,2023-01-01,Valetkevitchu,43
3,CUS_0x1184,Cohenq,49.0,963-76-2464,Lawyer,2023-01-01,Cohenq,49
4,CUS_0x1297,Edwardsz,46.0,Unknown,Manager,2023-01-01,Edwardsz,46
5,CUS_0x12fb,Jeb Blountz,35.0,006-29-8039,Doctor,2023-01-01,Jeb Blountz,35
6,CUS_0x1325,Philipz,27.0,918-42-0845,Accountant,2023-01-01,Philipz,27
7,CUS_0x1341,Zhuk,47.0,155-02-9593,Mechanic,2023-01-01,Zhuk,47
8,CUS_0x1375,Stanley Whitev,15.0,841-27-0645,Lawyer,2023-01-01,Stanley Whitev,15
9,CUS_0x13a8,Baileyz,NaN,918-56-2439,Journalist,2023-01-01,Baileyz,41
